<a href="https://colab.research.google.com/github/satyajit1288/DAA_Algorithms/blob/main/Cosine_attack_demonstration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download the BMI Dataset from Kaggle. This will Demonstrate the cosine attack using  'BMI' dataset. This data frame contains the following columns:

1) Gender : Male / Female

2) Height : Number (cm)

3) Weight : Number (Kg)

4) Index : (
0 - Extremely Weak ,
1 - Weak ,
2 - Normal ,
3 - Overweight ,
4 - Obesity ,
5 - Extreme Obesity )

In [1]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
# Based on the dataset handle, assuming a CSV file exists with a similar name.
file_path = "bmi.csv" # Replace with the actual file name if different

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "yasserh/bmidataset",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

/tmp/ipython-input-1655670895.py:11: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


100%|██████████| 8.12k/8.12k [00:00<00:00, 8.51MB/s]


Get the training data only and name it as X (only Gender , Height and Weight) and Repair X such that Feature : 'Gender'  (male = 1 and female = -1)

In [2]:
# Features (training data): just Gender, Height, Weight and convert it to numpy
import numpy as np
X = df[['Gender', 'Height', 'Weight']].copy()
X['Gender'] = np.where(X['Gender'].str.strip().str.lower() == 'male', 1, -1)

X_new = X.to_numpy()


**Phase - 1**

**Step-1**

i) select 3 buffer points as list1 (without replacement , known buffer vectors)

ii) select one data point as list2 (without replacement , hidden data point)

iii) find the cosine similarities between list_1 and list_2 and store them in list_3

In [3]:
import numpy as np

# i) choose 3 samples (store it in list_1)
indices = np.random.choice(X_new.shape[0], 3, replace=False)
list_1 = X_new[indices]

# ii) choose 1 sample (store it in list_2) --------> ( Unknown data point )
index = np.random.choice(X_new.shape[0], 1, replace=False)[0]
list_2 = X_new[index]
print("Indices for three samples:", indices)
print("Three samples (list_1):\n", list_1)
print("\nIndex for one sample:", index)
print("One sample (list_2):\n", list_2)

#Normalize buffer points(list_1) and make them unit vector
axes = np.array([v / np.linalg.norm(v) for v in list_1])

norm_list_2 = np.linalg.norm(list_2)

# iii) compute cosine similarity between list_2(unknown data point) with each row in list_1
list_3 = np.array([
    np.dot(row, list_2) / (np.linalg.norm(row) * norm_list_2)
    for row in list_1
])

print("\nCosine similarities (list_3):\n", list_3)

Indices for three samples: [  6   7 271]
Three samples (list_1):
 [[  1 147  92]
 [  1 154 111]
 [  1 152 103]]

Index for one sample: 176
One sample (list_2):
 [ -1 170 102]

Cosine similarities (list_3):
 [0.99976503 0.99641122 0.99842542]


**Step_2**

i) define a function name as "compute_direction"

input - list1 , list3

output - direction of the intersection

**This function will give a Unit vector as an output which is nothing but the direction of intersection between three buffer points**

In [4]:
import numpy as np
from itertools import product

def compute_direction(axes, list_3):
    """
    Compute the intersection direction of three cones.

    Args:
        list_1 (ndarray): shape (3, d). The three axis vectors (d = dimension, here 3).
        list_3 (ndarray): shape (3,). Cosine similarity values = cos(theta_i).
        tol (float): tolerance for unit norm check.

    Returns:
        direction (ndarray): a valid unit direction vector (if found).
    """

    # Step 1: Get matrix A as the normalized vector of list_1
    A = axes

    #step-2
    # Only for all positive and all negative sign combinations
    sign_combos = np.array([[1,1,1], [-1,-1,-1]])

    for signs in sign_combos:
        s = np.array(signs)
        b_s = s * list_3

        # Solve A u = b_s
        try:
            u = np.linalg.solve(A, b_s)
        except np.linalg.LinAlgError:
            continue  # skip if singular

        # Step 4: Check if u is unit vector (≈ 1)
        norm_u = np.linalg.norm(u)

        if round(norm_u, 10) == 1.0:
          return u # return the first valid direction

    raise ValueError("No valid unit direction found for given inputs.")

In [5]:
# find the direction
# pass axes (normalized vector of list_1) and list_3
direction = compute_direction(axes , list_3)
print("Intersection direction vector:\n", direction)


Intersection direction vector:
 [-0.00504401  0.85748202  0.51448921]


**Phase - 2**

**Step_3**

i) Get the direction in form of(x_1 , x_2 , x_3) , so make it (x_1/x_1 , x_2/x_1 , x_3/x-1)

ii) direction_new = (x_1/x_1 , x_2/x_1 , x_3/x_1)

ii) now compare direction new to normlized list_2 vector

In [6]:
x1, x2, x3 = direction

direction_new = np.array([x1 / x1, x2 / x1, x3 / x1])

if(direction_new[1]<0 and direction_new[2]<0):
  direction_new = -1*direction_new

print("direction_new:", direction_new)

direction_new: [ -1.         169.99999983 101.9999999 ]


In [7]:
# check if reconstructed vector from the direction vector(unit norm) is same as the unknown data point or not

if np.array_equal(np.round(direction_new, 5), np.round(list_2, 5)):
  print("The reconstructed data point is same as hidden data point")

The reconstructed data point is same as hidden data point
